In [23]:
import pandas as pd
import numpy as np
import re
import gensim
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
from gensim import corpora, models, similarities
import pyLDAvis
from pprint import pprint
import pickle 
import os
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import random
from gensim.models.ldamodel import LdaModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import MmCorpus
import mymodule

In [24]:
data = pd.read_csv('dataset/DatasetLegal.csv')
str_answer = data['answer'].astype(str)
str_answer = str_answer.map(lambda x: re.sub('[,.!?#/]', '', x))
str_question = data['question'].astype(str)
str_question = str_question.map(lambda x: re.sub('[,.!?*/]', '', x))

In [25]:
sentense_token = []
for i in range(len(str_answer)):
  sentense_token.append(str_question[i])
  sentense_token.append(str_answer[i])

train_data = sentense_token[:13062]
test_data = sentense_token[13062:]

In [26]:
word_token_answer = []
for sentense in train_data:
  word = word_tokenize(sentense, engine='newmm')
  word_token_answer.append(word)

In [27]:
stopwords = list(thai_stopwords())
read_stopwords = pd.read_csv('dataset/add_stopwords.csv')
add_stopwords = read_stopwords['stopword'].values.tolist()
processed_answer = []
for sentense in word_token_answer:
  each_sentense = []
  for word in sentense:
    if(word not in stopwords + add_stopwords):
      each_sentense.append(word)
  processed_answer.append(each_sentense)
print(processed_answer[0][:8])

['มีเรื่อง', 'ปรึกษา', 'ครอบครัว', 'สินสมรส', 'สินส่วนตัว', 'การทราบ', 'ข้อเท็จจริง', 'วิธีการ']


In [28]:
id2word = corpora.Dictionary(processed_answer)
# print(id2word)
with open('model/id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)

2023-04-16 13:50:40.309 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary<0 unique tokens: []>
2023-04-16 13:50:41.862 INFO    gensim.corpora.dictionary: adding document #10000 to Dictionary<14628 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...>
2023-04-16 13:50:42.263 INFO    gensim.corpora.dictionary: built Dictionary<16502 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...> from 13062 documents (total 654776 corpus positions)
2023-04-16 13:50:42.267 INFO    gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary<16502 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...> from 13062 documents (total 654776 corpus positions)", 'datetime': '2023-04-16T13:50:42.267619', 'gensim': '4.3.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [29]:
corpus = []
for text in processed_answer:
  vec = id2word.doc2bow(text)
  corpus.append(vec)
MmCorpus.serialize('model/corpus.mm', corpus)

2023-04-16 13:50:43.300 INFO    gensim.corpora.mmcorpus: storing corpus in Matrix Market format to model/corpus.mm
2023-04-16 13:50:43.306 INFO    gensim.matutils: saving sparse matrix to model/corpus.mm
2023-04-16 13:50:43.308 INFO    gensim.matutils: PROGRESS: saving document #0
2023-04-16 13:50:43.404 INFO    gensim.matutils: PROGRESS: saving document #1000
2023-04-16 13:50:43.498 INFO    gensim.matutils: PROGRESS: saving document #2000
2023-04-16 13:50:43.592 INFO    gensim.matutils: PROGRESS: saving document #3000
2023-04-16 13:50:43.697 INFO    gensim.matutils: PROGRESS: saving document #4000
2023-04-16 13:50:43.789 INFO    gensim.matutils: PROGRESS: saving document #5000
2023-04-16 13:50:43.887 INFO    gensim.matutils: PROGRESS: saving document #6000
2023-04-16 13:50:43.981 INFO    gensim.matutils: PROGRESS: saving document #7000
2023-04-16 13:50:44.083 INFO    gensim.matutils: PROGRESS: saving document #8000
2023-04-16 13:50:44.182 INFO    gensim.matutils: PROGRESS: saving docu

In [30]:
num_topics = 6
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=100,
                                       chunksize=800,
                                       passes=5,
                                       alpha=0.9,
                                       eta=0.5
                                       )
with open('model/lda_model.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

2023-04-16 13:50:44.698 INFO    gensim.models.ldamodel: using serial LDA version on this node
2023-04-16 13:50:44.727 INFO    gensim.models.ldamodel: running online (multi-pass) LDA training, 6 topics, 5 passes over the supplied corpus of 13062 documents, updating model once every 800 documents, evaluating perplexity every 8000 documents, iterating 100x with a convergence threshold of 0.001000
2023-04-16 13:50:44.729 INFO    gensim.models.ldamodel: PROGRESS: pass 0, at document #800/13062
2023-04-16 13:50:47.537 INFO    gensim.models.ldamodel: merging changes from 800 documents into a model of 13062 documents
2023-04-16 13:50:47.556 INFO    gensim.models.ldamodel: topic #2 (0.900): 0.014*"บุตร" + 0.011*"ที่ดิน" + 0.009*"มาตรา" + 0.009*"ฟ้อง" + 0.009*"สิทธิ" + 0.009*"ศาล" + 0.008*"กรณี" + 0.007*"เจ้าหนี้" + 0.007*"เงิน" + 0.006*"ทายาท"
2023-04-16 13:50:47.559 INFO    gensim.models.ldamodel: topic #5 (0.900): 0.015*"บริษัท" + 0.012*"บ้าน" + 0.009*"ที่ดิน" + 0.008*"สามี" + 0.008*"กรณี" + 

In [31]:
lda_model.show_topics(num_topics=6, num_words=10)

[(0,
  '0.028*"เงิน" + 0.020*"รถ" + 0.017*"สัญญา" + 0.017*"บาท" + 0.015*"ฟ้อง" + 0.014*"จ่าย" + 0.013*"ขาย" + 0.012*"ธนาคาร" + 0.012*"ซื้อ" + 0.012*"เจ้าหนี้"'),
 (1,
  '0.028*"บ้าน" + 0.021*"แม่" + 0.019*"พ่อ" + 0.017*"ลูก" + 0.016*"แฟน" + 0.012*"สามี" + 0.011*"ชื่อ" + 0.009*"ทราบ" + 0.008*"ภรรยา" + 0.007*"รบกวน"'),
 (2,
  '0.041*"ที่ดิน" + 0.028*"บุตร" + 0.025*"มาตรา" + 0.024*"สามี" + 0.024*"สิทธิ" + 0.016*"ปพพ" + 0.015*"บิดา" + 0.014*"ทรัพย์สิน" + 0.012*"จดทะเบียน" + 0.012*"แบ่ง"'),
 (3,
  '0.038*"ศาล" + 0.016*"โจทก์" + 0.015*"จำเลย" + 0.013*"มาตรา" + 0.013*"ตำรวจ" + 0.011*"จำคุก" + 0.010*"ความผิด" + 0.009*"ฐาน" + 0.008*"กระทำความผิด" + 0.007*"คำพิพากษา"'),
 (4,
  '0.045*"นายจ้าง" + 0.038*"ลูกจ้าง" + 0.025*"สิทธิ" + 0.021*"มาตรา" + 0.018*"สัญญา" + 0.017*"จ่าย" + 0.014*"กรณี" + 0.012*"เลิกจ้าง" + 0.011*"คุ้มครอง" + 0.011*"ค่าจ้าง"'),
 (5,
  '0.044*"บริษัท" + 0.019*"ทำงาน" + 0.015*"แจ้ง" + 0.015*"วันที่" + 0.011*"งาน" + 0.011*"พนักงาน" + 0.010*"ถาม" + 0.009*"จ่าย" + 0.008*"ทนาย" + 0.0

In [32]:
topic_dict = {
    0 : "Contract",
    1 : "Labor",
    2 : "Undefine :)",
    3 : "Personal Right",
    4 : "Family",
    5 : "Succession",
}

with open('model/topic_dict.pkl', 'wb') as f:
    pickle.dump(topic_dict, f)


In [33]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

LDAvis_data_filepath
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

c:\Users\User\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.115118 -0.073428       1        1  20.859513
1     -0.212203  0.150076       2        1  18.828705
2      0.157911  0.225067       3        1  17.305619
3      0.119315 -0.007158       4        1  15.699394
5     -0.146739 -0.135885       5        1  15.480144
4      0.196835 -0.158672       6        1  11.826624, topic_info=          Term         Freq        Total Category  logprob  loglift
394     บริษัท  4571.000000  4571.000000  Default  30.0000  30.0000
235     ที่ดิน  5280.000000  5280.000000  Default  29.0000  29.0000
657    นายจ้าง  3488.000000  3488.000000  Default  28.0000  28.0000
659    ลูกจ้าง  2929.000000  2929.000000  Default  27.0000  27.0000
28         ศาล  5266.000000  5266.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
566          บ   440.485545   743.157573   Topic6  -5.1694   1.6118
606    ดังนั้น   569.948018  1293.323413   Topic6  -4.9117   1.3154
267      ทำงาน   677.402238  2627.556961   Topic6  -4.7390   0.7793
160     บังคับ   529.302278  1600.052725   Topic6  -4.9857   1.0286
62   ตามกฎหมาย   498.488828  1512.166450   Topic6  -5.0457   1.0251

[381 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.041797  กฎหมาย
0         2  0.000597  กฎหมาย
0         3  0.283626  กฎหมาย
0         4  0.214361  กฎหมาย
0         5  0.007165  กฎหมาย
...     ...       ...     ...
2837      2  0.003996       ”
2837      3  0.003996       ”
2837      4  0.003996       ”
2837      5  0.007993       ”
2837      6  0.979082       ”

[1298 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 6, 5])

The sections below working on lda model in question part

In [44]:
sentense_token_question = []
question_corpus = []
for each_question in str_question:
  sentense_token_question.append(each_question)
question_data = sentense_token_question
print(len(question_data))
topic_question = []
for sentense in question_data:
  processed_question = mymodule.preprocess(sentense)
  question_corpus.append(processed_question)
  topic = lda_model.get_document_topics(processed_question)
  topic_question.append(topic)

7031


In [45]:
with open('model/question_lda.pkl', 'wb') as f:
    pickle.dump(topic_question, f)
MmCorpus.serialize('model/question_corpus.mm', question_corpus)

2023-04-16 14:14:08.320 INFO    gensim.corpora.mmcorpus: storing corpus in Matrix Market format to model/question_corpus.mm
2023-04-16 14:14:08.323 INFO    gensim.matutils: saving sparse matrix to model/question_corpus.mm
2023-04-16 14:14:08.325 INFO    gensim.matutils: PROGRESS: saving document #0
2023-04-16 14:14:08.486 INFO    gensim.matutils: PROGRESS: saving document #1000
2023-04-16 14:14:08.701 INFO    gensim.matutils: PROGRESS: saving document #2000
2023-04-16 14:14:08.860 INFO    gensim.matutils: PROGRESS: saving document #3000
2023-04-16 14:14:09.012 INFO    gensim.matutils: PROGRESS: saving document #4000
2023-04-16 14:14:09.155 INFO    gensim.matutils: PROGRESS: saving document #5000
2023-04-16 14:14:09.323 INFO    gensim.matutils: PROGRESS: saving document #6000
2023-04-16 14:14:09.436 INFO    gensim.matutils: PROGRESS: saving document #7000
2023-04-16 14:14:09.442 INFO    gensim.matutils: saved 7031x16502 matrix, density=0.213% (247117/116025562)
2023-04-16 14:14:09.445 I

The sections below are in the process of being updated with new documents.

In [46]:
new_doc = random.choice(test_data)
print(new_doc)
test_doc = mymodule.preprocess(new_doc)

อยากทราบว่าถ้าให้คนอื่นที่ไม่มีอำนาจในการลงลายมือชื่อและออกใบเสร็จรับเงินแทนจะมีความผิดฐานปลอมแปลงเอกสารหรือเปล่าโดยที่คนที่ลงลายมือชื่อในใบเสร็จรับเงินนั้นไม่ได้เป็นเจ้าหน้าที่ในบริษัทด้วยจะมีความผิดอะไรอีกไหม


In [37]:
with open('model/question_lda.pkl', 'rb') as f:
    topic_question = pickle.load(f)
new_doc_topics = lda_model.get_document_topics(test_doc)
new_doc_topics_dict = {topic_dict[topic]: prob for topic, prob in new_doc_topics}
print(new_doc_topics_dict)

{'Undefine :)': 0.014627128, 'Personal Right': 0.95772624, 'Family': 0.016687682}


In [43]:
corpus = question_corpus
data = pd.read_csv('dataset/DatasetLegal.csv')
with open('model/id2word.pkl', 'rb') as f:
  id2word = pickle.load(f)
corpus_lda = lda_model[corpus]
index = similarities.MatrixSimilarity(corpus_lda, num_features=len(id2word))
sims = index[new_doc_topics]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(f"Topic distribution for new document : {new_doc_topics}\n{new_doc}\n")
for doc_id, similarity in sims_sorted[:5]:
    print(f"Document ID: {doc_id}, Similarity score: {similarity}")
    print(data.answer[doc_id])
    print("Topic distribution for similar document : ")
    for num, dis in corpus_lda[doc_id]:
      print(f"\t({topic_dict.get(num)}, {'%.5f' %dis})")

2023-04-16 14:09:34.404 INFO    gensim.similarities.docsim: creating matrix with 7031 documents and 16502 features


Topic distribution for new document : [(2, 0.014627128), (3, 0.95772624), (4, 0.016687682)]
ตามพรบยาเสพติดให้โทษพศได้กำหนดให้ยาบ้าจัดเป็นยาเสพติดให้โทษชนิดร้ายแรงประเภทเช่นเดียวกับเฮโรอีนตามมาตราแห่งพรบดังกล่าวและมาตราได้บัญญัติถึงความผิดและกำหนดโทษเอาไว้ว่าผู้ใดมีไว้ครอบครองซึ่งยาเสพติดให้โทษในประเภทโดยไม่ได้รับอนุญาตและมีปริมาณคำนวณเป็นสารบริสุทธิ์หรือมีหน่วยการใช้หรือมีน้ำหนักสุทธิไม่ถึงปริมาณที่กำหนดตามมาตราวรรคต้องระวางโทษจำคุกตั้งแต่ปีถึงปีหรือปรับตั้งแต่บาทถึงบาทหรือทั้งจำทั้งปรับเพราะฉะนั้นหากท่านถูกฟ้องคดีในความผิดดังกล่าวต่อศาลแล้วท่านจะให้การเบิกความรับสารภาพต่อชั้นศาลกรณีย่อมเป็นการให้ความรู้แก่ศาลอันเป็นประโยชน์แก่การพิจารณาเข้าเหตุบรรเทาโทษที่ศาลจะพิพากษาลงโทษท่านไม่เกินกึ่งหนึ่งของโทษที่จะลงแก่ท่านตามความผิดนั้นก็ได้เมื่อศาลเห็นเป็นการสมควรตามปอมาตราอนึ่งการที่ศาลจะลงโทษจำคุกหรือปรับแก่ท่านนั้นย่อมเป็นเรื่องการใช้ดุลพินิจของศาลในการกำหนดโทษที่จะลงแก่ท่านเองโดยพิจารณาประกอบกับกรณีที่ท่านเคยต้องคำพิพากษาถึงที่สุดให้ลงโทษจำคุกสำหรับความผิดตามพรบยาเสพติดให้โทษพศซึ่งท่านได้กร

In [42]:
corpus_lda = lda_model[topic_question]
index = similarities.MatrixSimilarity(corpus_lda)
sims = index[new_doc_topics]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(f"Topic distribution for new document : {new_doc_topics}\n{new_doc}\n")
for doc_id, similarity in sims_sorted[:5]:
    print(f"Document ID: {doc_id}, Similarity score: {similarity}")
    print(data.answer[doc_id])
    print("Topic distribution for similar document : ")
    for num, dis in corpus_lda[doc_id]:
      print(f"\t({topic_dict.get(num)}, {'%.5f' %dis})")

2023-04-16 14:07:56.729 WARNING gensim.similarities.docsim: scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2023-04-16 14:08:01.639 INFO    gensim.similarities.docsim: creating matrix with 7031 documents and 6 features


Topic distribution for new document : [(2, 0.014627128), (3, 0.95772624), (4, 0.016687682)]
ตามพรบยาเสพติดให้โทษพศได้กำหนดให้ยาบ้าจัดเป็นยาเสพติดให้โทษชนิดร้ายแรงประเภทเช่นเดียวกับเฮโรอีนตามมาตราแห่งพรบดังกล่าวและมาตราได้บัญญัติถึงความผิดและกำหนดโทษเอาไว้ว่าผู้ใดมีไว้ครอบครองซึ่งยาเสพติดให้โทษในประเภทโดยไม่ได้รับอนุญาตและมีปริมาณคำนวณเป็นสารบริสุทธิ์หรือมีหน่วยการใช้หรือมีน้ำหนักสุทธิไม่ถึงปริมาณที่กำหนดตามมาตราวรรคต้องระวางโทษจำคุกตั้งแต่ปีถึงปีหรือปรับตั้งแต่บาทถึงบาทหรือทั้งจำทั้งปรับเพราะฉะนั้นหากท่านถูกฟ้องคดีในความผิดดังกล่าวต่อศาลแล้วท่านจะให้การเบิกความรับสารภาพต่อชั้นศาลกรณีย่อมเป็นการให้ความรู้แก่ศาลอันเป็นประโยชน์แก่การพิจารณาเข้าเหตุบรรเทาโทษที่ศาลจะพิพากษาลงโทษท่านไม่เกินกึ่งหนึ่งของโทษที่จะลงแก่ท่านตามความผิดนั้นก็ได้เมื่อศาลเห็นเป็นการสมควรตามปอมาตราอนึ่งการที่ศาลจะลงโทษจำคุกหรือปรับแก่ท่านนั้นย่อมเป็นเรื่องการใช้ดุลพินิจของศาลในการกำหนดโทษที่จะลงแก่ท่านเองโดยพิจารณาประกอบกับกรณีที่ท่านเคยต้องคำพิพากษาถึงที่สุดให้ลงโทษจำคุกสำหรับความผิดตามพรบยาเสพติดให้โทษพศซึ่งท่านได้กร